In [ ]:
from pprint import pprint

In [ ]:
from ipy_table import make_table

def make_list_table(items):
    return make_table([[repr(item)] for item in items])

In [ ]:
%%HTML
<style>td { font-family: monospace; }</style>

In [ ]:
content = '''


first  second      third         fourth  fifth
val 1  number two   do()                 fourth missing
val 2               go()  fifth missing


'''

In [ ]:
lines = content.split('\n')
make_list_table(lines)

In [ ]:
from itertools import dropwhile

def is_blank(s):
    return not s.strip()

without_leading_blanks = list(dropwhile(is_blank, lines))
make_list_table(without_leading_blanks)

In [ ]:
from itertools import takewhile
import string

body = list(takewhile(string.strip, without_leading_blanks))
make_list_table(body)

In [ ]:
max_length = max(len(line) for line in lines)
max_length

In [ ]:
padded_lines = ['{line:<{length}s}'.format(line=line, length=max_length)
                for line in body]
make_list_table(padded_lines)

In [ ]:
header = padded_lines[0]
make_list_table([header])

In [ ]:
data_lines = padded_lines[1:]
make_list_table(data_lines)

In [ ]:
def all_same(lines, position, character):
    return all(line[position] == character for line in padded_lines[1:])

seps = ''.join('|' if all_same(data_lines, position, character) else ' '
               for position, character in enumerate(header))
make_list_table(padded_lines + [seps])

In [ ]:
import re

spans = [m.span() for m in re.finditer(r'\|{2,}', seps)]
pprint(spans)
make_table(spans)

In [ ]:
starts = [0] + [span[1] for span in spans]
pprint(starts)

In [ ]:
ends = [span[0] for span in spans] + [None]
pprint(ends)

In [ ]:
def pick_columns(line, starts, ends):
    return [line[start:end].strip() for start, end in zip(starts, ends)]

result = [pick_columns(line, starts, ends) for line in padded_lines]
pprint(result) ; make_table(result)

In [ ]:
import re
from itertools import takewhile, dropwhile
from ipy_table import make_table
import string

def all_same(lines, position, character):
    return all(line[position] == character for line in lines[1:])

def is_blank(s):
    return not s.strip()

def pick_columns(line, starts, ends):
    return [line[start:end].strip() for start, end in zip(starts, ends)]

def parse_table(content):
    lines = content.split('\n')
    without_leading_blanks = dropwhile(is_blank, lines)
    body = takewhile(string.strip, without_leading_blanks)
    max_length = max(len(line) for line in lines)
    padded_lines = ['{line:<{length}s}'.format(line=line, length=max_length)
                    for line in body]
    header = padded_lines[0]
    data_lines = padded_lines[1:]
    seps = ''.join('|' if all_same(data_lines, position, character) else ' '
                   for position, character in enumerate(header))
    spans = [m.span() for m in re.finditer(r'\|{2,}', seps)]
    starts = [0] + [span[1] for span in spans]
    ends = [span[0] for span in spans] + [None]
    return [pick_columns(line, starts, ends) for line in padded_lines]

In [ ]:
parse_table(content)

In [ ]:
def iterate_cells(table):
    column_titles = table[0][1:]
    rows = table[1:]
    for row in rows:
        row_title = row[0]
        for column_num, cell in enumerate(row[1:]):
            column_title = column_titles[column_num]
            yield row_title, column_title, cell

In [ ]:
cells = list(iterate_cells(result))
pprint(cells) ; make_table(cells)

In [ ]:
import pytest

In [ ]:
def pytest_mark_parametrize2d(signature, matrix, **namespace):
    # Get globals and locals from the calling scope
    eval_namespace = inspect.currentframe().f_back.f_globals.copy()
    eval_namespace.update(inspect.currentframe().f_back.f_locals)
    eval_namespace.update(namespace)
    cells = iterate_cells(parse_table(matrix))
    rows = [(eval(row, eval_namespace),
             eval(col, eval_namespace),
             eval(cell, eval_namespace))
            for row, col, cell in cells]
    return pytest.mark.parametrize(signature, rows)

In [ ]:
i = iter([1,2,3])

In [ ]:
ctx = {}

In [ ]:
exec('a=1\nb=2\nc=3', ctx)

In [ ]:
ctx['c']

In [ ]:
tuple(x for x in [1,2,3])

In [ ]:
def split_by_blank_lines(content):
    lines = content.split('\n')
    while True:
        without_leading_blanks = dropwhile(is_blank, lines)
        yield list(takewhile(string.strip, without_leading_blanks))

In [ ]:
for i in split_by_blank_lines('a\n\nb\n\nc\n\nd\n'):
    print i